# Neural Network

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from pathlib import Path

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Set log level to reduce verbosity

import tensorflow as tf

# Restrict TensorFlow to only allocate 12GB of memory on the first GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=12288)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

tf.config.optimizer.set_experimental_options({"auto_mixed_precision": False})

import math

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB3

from sklearn.metrics import *

from tqdm.notebook import tqdm

import pickle


2024-07-20 21:11:19.816990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 21:11:19.817113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 21:11:19.948821: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


1 Physical GPUs, 1 Logical GPUs


## Configurations

In [ ]:
## Configure    
N_TOP = 25

IMG_SIZE = 128

CUTOUT_COUNT = 5
CUTOUT_P = 0.75
CUTOUT_MASKSIZE = 0.2

CROP_SIZE = 0.5
CROP_P = 0.75

LR_START = 1e-06
LR_MAX = 5e-5
LR_MIN = 1e-9
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 25
EPOCHS = 50

TF_VERBOSE = 2

EFFNET_DROP_CONNCET_RATE = 0.35
HEAD_DENSE_LAYER = 512
HEAD_DROPOUT = 0.25

ds_path = Path(os.getcwd()).parent / 'Backend/Dataset'
output_path = Path(os.getcwd()).parent / 'Backend/Output'
batch_size = 16

## Necessary Functions

In [ ]:
def train_preprocess_image(file_path, label):
    return preprocess_image(file_path, label, augmentation = True, rescale = True, 
                            crop_size = CROP_SIZE, crop_p = CROP_P,
                            cutout_count=CUTOUT_COUNT, 
                            cutout_p = CUTOUT_P,  
                            cutout_masksize= CUTOUT_MASKSIZE)

def test_preprocess_image(file_path, label):
    return preprocess_image(file_path, label, augmentation = False, rescale = True)


In [ ]:
def configure_ds_for_performance(ds, batch_size, shuffle=True, buffer_size=1000):
    if shuffle:
        ds = ds.shuffle(buffer_size=buffer_size)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def create_ds(df, ds_path):
    filenames = [ f"{ds_path}/{filename}" for filename in  df["filename"].values]
    labels = [artist for artist in df["artist_class"]]
    ds = tf.data.Dataset.from_tensor_slices((filenames, labels))
    return ds

def create_ds_artist(df, ds_path):
    filenames = [ f"{ds_path}/{filename}" for filename in  df["filename"].values]
    labels = [artist for artist in df["artist_class"]]
    ds = tf.data.Dataset.from_tensor_slices((filenames, labels))
    return ds

def create_ds_genre(df, ds_path):
    filenames = [ f"{ds_path}/{filename}" for filename in  df["filename"].values]
    labels = [genre for genre in df["genre_class"]]
    ds = tf.data.Dataset.from_tensor_slices((filenames, labels))
    return ds

In [ ]:
def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        decay_total_epochs = EPOCHS - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1
        decay_epoch_index = epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS
        phase = math.pi * decay_epoch_index / decay_total_epochs
        cosine_decay = 0.5 * (1 + math.cos(phase))
        lr = (LR_MAX - LR_MIN) * cosine_decay + LR_MIN
    return lr

# Plot the training graph
def plot_training(history):
    acc = history['accuracy']
    val_acc = history['val_accuracy']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(len(acc))

    fig, axes = plt.subplots(1, 2, figsize=(20,5))
    
    axes[0].plot(epochs, acc, 'r-', label='Training Accuracy')
    axes[0].plot(epochs, val_acc, 'b--', label='Validation Accuracy')
    axes[0].set_title('Training and Validation Accuracy')
    axes[0].legend(loc='best')

    axes[1].plot(epochs, loss, 'r-', label='Training Loss')
    axes[1].plot(epochs, val_loss, 'b--', label='Validation Loss')
    axes[1].set_title('Training and Validation Loss')
    axes[1].legend(loc='best')
    
    plt.show()

In [ ]:
def resize(image, size):
    return tf.image.resize(image, [size, size])

def dropout(img, size, p, count, cutout_masksize):
    p = tf.cast(tf.random.uniform([], 0, 1) < p, tf.int32)
    if (p == 0) | (count == 0) | (cutout_masksize == 0):
        return img

    for _ in range(count):
        x = tf.cast(tf.random.uniform([], 0, size), tf.int32)
        y = tf.cast(tf.random.uniform([], 0, size), tf.int32)
        width = tf.cast(cutout_masksize * size, tf.int32)
        ya = tf.maximum(0, y - width // 2)
        yb = tf.minimum(size, y + width // 2)
        xa = tf.maximum(0, x - width // 2)
        xb = tf.minimum(size, x + width // 2)
        one = img[ya:yb, 0:xa, :]
        two = tf.zeros([yb - ya, xb - xa, 3])
        three = img[ya:yb, xb:size, :]
        middle = tf.concat([one, two, three], axis=1)
        img = tf.concat([img[0:ya, :, :], middle, img[yb:size, :, :]], axis=0)
    img = tf.reshape(img, [size, size, 3])
    return img

def preprocess_image(file_path, label, augmentation=False, rescale=True, crop_size=0.5, crop_p=0.75, cutout_count=5, cutout_p=0.75, cutout_masksize=0.2):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels=3)

    if augmentation:
        crop_p = tf.cast(tf.random.uniform([], 0, 1) < crop_p, tf.float32)
        r = 1.0 + crop_size * np.random.random() * crop_p
        img = resize(img, size=int(IMG_SIZE * r))
        img = tf.image.random_crop(img, size=[IMG_SIZE, IMG_SIZE, 3])
        img = tf.image.random_flip_left_right(img)
        if cutout_count > 0:
            img = dropout(img, size=IMG_SIZE, p=cutout_p, count=cutout_count, cutout_masksize=cutout_masksize)
    else:
        img = resize(img, size=IMG_SIZE)

    if rescale:
        img = tf.cast(img, tf.float32) / 255.0

    return img, label


In [ ]:
def build_model(n_classes, return_feature_extractor_model=False):
    inp = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model = tf.keras.applications.EfficientNetB3(include_top=False, weights=None, input_tensor=inp)  # Set weights to None

    # Add layers at the end
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    if return_feature_extractor_model:
        feat_extractor1 = x
        
    x = tf.keras.layers.Dense(HEAD_DENSE_LAYER, name="head_dense_512", kernel_initializer='he_uniform')(x)
    
    if return_feature_extractor_model:
        feat_extractor2 = x

    x = tf.keras.layers.Dropout(HEAD_DROPOUT)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    if return_feature_extractor_model:
        feat_extractor = tf.keras.layers.concatenate([feat_extractor1, feat_extractor2])

    output = tf.keras.layers.Dense(n_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=base_model.input, outputs=output)
    if return_feature_extractor_model:
        model_fe = tf.keras.Model(inputs=base_model.input, outputs=feat_extractor)
        return model, model_fe
    else:
        return model


In [ ]:
def check_files_exist(df_classes, base_path):
    """
    Check if all files listed in df_classes are present in the base_path directory.
    
    Args:
    df_classes (pd.DataFrame): DataFrame containing the file paths.
    base_path (str): The base directory where the files should be located.
    
    Returns:
    list: A list of missing files.
    """
    missing_files = []
    for file_path in df_classes['filename'].values:
        if not os.path.exists(os.path.join(base_path, file_path)):
            missing_files.append(file_path)
    return missing_files

# Example usage
df_classes = pd.read_csv(f"{ds_path}/classes.csv")  # Update with your actual path
missing_files = check_files_exist(df_classes, ds_path)

if missing_files:
    print(f"Missing files: {missing_files}")
else:
    print("All files exist.")


Missing files: ['Art_Nouveau_Modern/eugã¨ne-grasset_a-la-place-clichy.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_abricotine.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_affiche-pour-the-century-magazine-napol-on.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_anxi-t-1897.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_bonne-nouvelle.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_encre-l-marquet.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_esclarmonde.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_froideur.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_girl-in-the-garden.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_grafton-gallery-from-les-affiche-illustrees-1897.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_harper-s-magazine.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_jalousie.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_joan-of-arc-1894.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_la-belle-jardiniere-april-1896.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_la-belle-jardiniere-august-1896.jpg', 'Art_Nouveau_Modern/eugã¨ne-grasset_la-bell

# Predict Artist

In [224]:
df_artist = df_classes.groupby(["subset","artist"]).size().reset_index()
df_artist.columns = ["subset", "artist", "size"]
df_artist = df_artist.pivot(index="artist",columns="subset", values="size")
df_artist = df_artist.reset_index()
df_artist = df_artist.fillna(0).sort_values(by="train", ascending=False)
df_artist["train"] = df_artist["train"].astype(np.int16)
df_artist["test"] = df_artist["test"].astype(np.int16)
df_artist["uncertain artist"] = df_artist["uncertain artist"].astype(np.int16)

df_artist=df_artist.sort_values(by="train", ascending=False)
N_valid = df_artist.shape[0] - (df_artist['train']<=1).sum()

top_artists = df_artist["artist"].values[:N_valid]
display(df_artist.head(N_TOP))

df_all_artist = df_classes[df_classes["artist"].isin(top_artists)].reset_index(drop = True)
le_artist = LabelEncoder()
df_all_artist["artist_class"] = le_artist.fit_transform(df_all_artist["artist"].values)
class_names_artist = le_artist.classes_

with open('artist_class_names.pickle', 'wb') as handle:
    pickle.dump(class_names_artist, handle)
    
df_train_artist = df_all_artist.query("subset == 'train'").reset_index(drop = True)
df_test_artist = df_all_artist.query("subset == 'test'").reset_index(drop = True)

df_train_artist, df_valid_artist, y_train_artist, y_valid_artist =  train_test_split(df_train_artist, df_train_artist["artist"], 
                                                                   test_size=0.2, random_state=42, 
                                                                   stratify=df_train_artist["artist"])

print(f"train:{df_train_artist.shape[0]}, valid:{df_valid_artist.shape[0]}, test:{df_test_artist.shape[0]}")

df_artist = df_train_artist.groupby("artist_class").size().reset_index()
df_artist.columns = ["artist_class","size"]
df_artist["class_weight"] = df_artist["size"].sum()/(df_artist.shape[0]*df_artist["size"])

# Set class weights - assign higher weights to underrepresented classes
class_weights_artist = df_artist['class_weight'].to_dict()

subset,artist,test,train,uncertain artist
1074,vincent van gogh,378,1510,0
809,nicholas roerich,363,1453,0
898,pierre auguste renoir,280,1117,2
190,claude monet,267,1067,0
917,pyotr konchalovsky,185,739,0


NameError: name 'df_train_artist_artist' is not defined

In [ ]:
train_ds_artist = create_ds_artist(df_train_artist, ds_path)
train_ds_artist = train_ds_artist.map(train_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds_artist = configure_ds_for_performance(train_ds_artist,batch_size, shuffle=True)

valid_ds_artist = create_ds_artist(df_valid_artist, ds_path)
valid_ds_artist = valid_ds_artist.map(test_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
valid_ds_artist = configure_ds_for_performance(valid_ds_artist,batch_size, shuffle=True)

test_ds_artist = create_ds_artist(df_test_artist, ds_path)
test_ds_artist = test_ds_artist.map(test_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds_artist = configure_ds_for_performance(test_ds_artist,batch_size, shuffle=False)

In [ ]:
early_stop_artist = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=1, 
                           mode='auto', restore_best_weights=True)

model_chkpoint_artist = tf.keras.callbacks.ModelCheckpoint(
        filepath=str(output_path) + '/model_artist.weights.h5', monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')

reduce_lr_artist = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)

In [ ]:
n_epoch = EPOCHS
#model = build_model(N_TOP, return_feature_extractor_model = False)
#model = build_model(5, return_feature_extractor_model = False)
model_artist = build_model(N_valid)

optimizer = tf.keras.optimizers.Adam(learning_rate=LR_START)

model_artist.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

history_artist = model_artist.fit(train_ds_artist, 
                        validation_data=valid_ds_artist, 
                        epochs=n_epoch,
                        verbose=1,#TF_VERBOSE,
                        callbacks=[reduce_lr_artist, early_stop_artist, model_chkpoint_artist],
                        class_weight=class_weights_artist
                        )
plot_training(history_artist.history)

In [ ]:
score_artist = model_artist.evaluate(valid_ds_artist, verbose=TF_VERBOSE)
print(f"accuracy valid: {score_artist[1]:.5f}")

In [ ]:
y_pred_artist, y_true_artist = [], []
for (X,y) in tqdm(test_ds_artist): 
    y_pred_artist.append(model_artist.predict(X))
    y_true_artist.append(y.numpy())

y_true_artist = np.concatenate(y_true_artist)

y_pred_artist = np.concatenate(y_pred_artist)
y_pred_artist = np.argmax(y_pred_artist, axis=1)

score_artist = accuracy_score(y_true_artist, y_pred_artist)
print(f"accuracy test: {score_artist:.5f}")


In [ ]:
with open('artist_class_names.pickle', 'rb') as handle:
    artist_class_names = pickle.load(handle)
    
tick_labels = artist_class_names
fig, ax = plt.subplots(figsize=(15,15))
conf_matrix = confusion_matrix(y_true_artist, y_pred_artist, labels=np.arange(N_TOP))
conf_matrix = conf_matrix/np.sum(conf_matrix, axis=1)
sns.heatmap(conf_matrix, annot=True, fmt=".2f", square=True, cbar=False, 
            xticklabels=tick_labels, yticklabels=tick_labels,cmap=plt.cm.YlGn,
            ax=ax)
ax.set_ylabel('Actual')
ax.set_xlabel('Predicted')
ax.set_title('Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(y_true_artist, y_pred_artist, labels=np.arange(N_TOP), target_names=class_names))

In [ ]:
# Save model
model_artist.save('models/model_artist.h5')

# Predict Genre

In [18]:
df_classes = pd.read_csv(f"{ds_path}/classes.csv")

df_genre = df_classes.groupby(["subset","genre"]).size().reset_index()
df_genre.columns = ["subset", "genre", "size"]
df_genre = df_genre.pivot(index="genre",columns="subset", values="size")
df_genre = df_genre.reset_index()
df_genre = df_genre.fillna(0).sort_values(by="train", ascending=False)
df_genre["train"] = df_genre["train"].astype(np.int16)
df_genre["test"] = df_genre["test"].astype(np.int16)
df_genre["uncertain artist"] = df_genre["uncertain artist"].astype(np.int16)

df_genre=df_genre.sort_values(by="train", ascending=False)
N_valid = df_genre.shape[0] - (df_genre['train']<=1).sum()
N_TOP = N_valid

top_genres = df_genre["genre"].values[:N_TOP]#[:N_valid]
display(df_genre.head(N_TOP))

df_all_genre = df_classes[df_classes["genre"].isin(top_genres)].reset_index(drop = True)
# Filter the missing values
df_all_genre = df_all_genre[~df_all_genre['filename'].isin(missing_files)]

le_genre = LabelEncoder()
df_all_genre["genre_class"] = le_genre.fit_transform(df_all_genre["genre"].values)
class_names_genre = le_genre.classes_

with open('genre_class_names.pickle', 'wb') as handle:
    pickle.dump(class_names_genre, handle)
    
df_train_genre = df_all_genre.query("subset == 'train'").reset_index(drop = True)
df_test_genre = df_all_genre.query("subset == 'test'").reset_index(drop = True)

df_train_genre, df_valid_genre, y_train_genre, y_valid_genre =  train_test_split(
    df_train_genre, df_train_genre["genre"], test_size=0.2, random_state=42, stratify=df_train_genre["genre"])

print(f"train:{df_train_genre.shape[0]}, valid:{df_valid_genre.shape[0]}, test:{df_test_genre.shape[0]}")

df_genre = df_train_genre.groupby("genre_class").size().reset_index()
df_genre.columns = ["genre_class","size"]
df_genre["class_weight"] = df_genre["size"].sum()/(df_genre.shape[0]*df_genre["size"])

# Set class weights - assign higher weights to underrepresented classes
class_weights_genre = df_genre['class_weight'].to_dict()

subset,genre,test,train,uncertain artist
51,['Impressionism'],2576,10256,15
91,['Realism'],2118,8412,4
98,['Romanticism'],1384,5504,8
89,['Post Impressionism'],1245,5029,0
30,['Expressionism'],1278,5001,1
...,...,...,...,...
44,"['Impressionism', 'Fauvism']",0,2,0
18,"['Cubism', 'Abstract Expressionism']",1,2,0
104,"['Symbolism', 'Naive Art Primitivism']",0,2,0
56,"['Minimalism', 'Minimalism']",2,2,0


train:50738, valid:12685, test:15846


In [20]:
train_ds_genre = create_ds_genre(df_train_genre, ds_path)
train_ds_genre = train_ds_genre.map(train_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds_genre = configure_ds_for_performance(train_ds_genre,batch_size, shuffle=True)

valid_ds_genre = create_ds_genre(df_valid_genre, ds_path)
valid_ds_genre = valid_ds_genre.map(test_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
valid_ds_genre = configure_ds_for_performance(valid_ds_genre,batch_size, shuffle=True)

test_ds_genre = create_ds_genre(df_test_genre, ds_path)
test_ds_genre = test_ds_genre.map(test_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds_genre = configure_ds_for_performance(test_ds_genre,batch_size, shuffle=False)

In [21]:
early_stop_genre = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=1, 
                           mode='auto', restore_best_weights=True)

model_chkpoint_genre = tf.keras.callbacks.ModelCheckpoint(
        filepath=str(output_path) + '/model_genre.weights.h5', monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')

reduce_lr_genre = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)

In [ ]:
n_epoch = EPOCHS
#model = build_model(N_TOP, return_feature_extractor_model = False)
#model = build_model(5, return_feature_extractor_model = False)
model_genre = build_model(N_TOP)#(N_valid)

optimizer = tf.keras.optimizers.Adam(learning_rate=LR_START)

model_genre.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

history_genre = model_genre.fit(train_ds_genre, 
                        validation_data=valid_ds_genre, 
                        epochs=n_epoch,
                        verbose=1,#TF_VERBOSE,
                        callbacks=[reduce_lr_genre, early_stop_genre, model_chkpoint_genre],
                        class_weight=class_weights_genre
                        )
plot_training(history_genre.history)

Epoch 1/50
   2/3172 ━━━━━━━━━━━━━━━━━━━━ 3:58 75ms/step - accuracy: 0.0156 - loss: 40.3342         

W0000 00:00:1721511018.164425     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3172/3172 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.0089 - loss: 4.8625

W0000 00:00:1721511434.272208     102 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_loss improved from inf to 4.77992, saving model to ../output/model_genre.weights.h5
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 649s 158ms/step - accuracy: 0.0089 - loss: 4.8625 - val_accuracy: 0.0095 - val_loss: 4.7799 - learning_rate: 1.0000e-06
Epoch 2/50
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.0110 - loss: 4.8185
Epoch 2: val_loss improved from 4.77992 to 4.52391, saving model to ../output/model_genre.weights.h5
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 410s 127ms/step - accuracy: 0.0110 - loss: 4.8185 - val_accuracy: 0.0164 - val_loss: 4.5239 - learning_rate: 1.0800e-05
Epoch 3/50
3171/3172 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.0185 - loss: 4.4753
Epoch 3: val_loss improved from 4.52391 to 4.39399, saving model to ../output/model_genre.weights.h5
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 421s 131ms/step - accuracy: 0.0185 - loss: 4.4754 - val_accuracy: 0.0287 - val_loss: 4.3940 - learning_rate: 2.0600e-05
Epoch 4/50
3171/3172 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy:

In [ ]:
score_genre = model_genre.evaluate(valid_ds_genre, verbose=TF_VERBOSE)
print(f"accuracy valid: {score_genre[1]:.5f}")

In [ ]:
y_pred_genre, y_true_genre = [], []
for (X,y) in tqdm(test_ds_genre): 
    y_pred_genre.append(model_genre.predict(X))
    y_true_genre.append(y.numpy())

y_true_genre = np.concatenate(y_true_genre)

y_pred_genre = np.concatenate(y_pred_genre)
y_pred_genre = np.argmax(y_pred_genre, axis=1)

score_genre = accuracy_score(y_true_genre, y_pred_genre)
print(f"accuracy test: {score_genre:.5f}")


In [ ]:
with open('genre_class_names.pickle', 'rb') as handle:
    genre_class_names = pickle.load(handle)
    
tick_labels = genre_class_names
fig, ax = plt.subplots(figsize=(15,15))
conf_matrix = confusion_matrix(y_true_genre, y_pred_genre, labels=np.arange(N_TOP))
conf_matrix = conf_matrix/np.sum(conf_matrix, axis=1)
sns.heatmap(conf_matrix, annot=True, fmt=".2f", square=True, cbar=False, 
            xticklabels=tick_labels, yticklabels=tick_labels,cmap=plt.cm.YlGn,
            ax=ax)
ax.set_ylabel('Actual')
ax.set_xlabel('Predicted')
ax.set_title('Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(y_true_genre, y_pred_genre, labels=np.arange(N_TOP), target_names=class_names))

In [ ]:
# Save model
model_genre.save('models/model_genre.h5')